In [27]:
# Gerekli kütüphaneler
!pip install -q transformers sentencepiece torch pandas tqdm
import torch
import time
import pandas as pd
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from tqdm import tqdm
import subprocess
import os

# GPU kullanımını ölçme fonksiyonu
def log_gpu_usage(log_file='gpu_usage.log'):
    """GPU kullanımını arka planda loglar"""
    cmd = f'nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv -l 1 > {log_file}'
    process = subprocess.Popen(cmd, shell=True)
    return process

# Model ve tokenizer yükleme
model_name = "facebook/m2m100_418M"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"\n🔍 {model_name} modeli yükleniyor...")
start_load = time.time()

tokenizer = M2M100Tokenizer.from_pretrained(model_name, src_lang="en", tgt_lang="tr")
model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)

load_time = time.time() - start_load
print(f"✅ Yükleme tamamlandı ({load_time:.2f}s)")
print(f"🖥️ Cihaz: {device.upper()}")
print(f"📊 Model boyutu: {sum(p.numel() for p in model.parameters())/1e6:.1f}M parametre")

# Çeviri fonksiyonu (optimize edilmiş)
def translate_to_turkish(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.get_lang_id("tr"),
        max_length=512,
        num_beams=5,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Veri setini yükleme
print("\n📂 Veri seti yükleniyor...")
df = pd.read_csv('test_dataset_500_sentences.csv')
sentences = df['Sentence'].tolist()  # 'Sentence' sütunundaki tüm cümleler
print(f"Toplam {len(sentences)} cümle bulundu")

# GPU logunu başlat
gpu_process = log_gpu_usage()

# Performans ölçümü
print("\n⚡ Çeviri işlemi başlatılıyor...")
results = []
start_total = time.time()

for i, sentence in enumerate(tqdm(sentences, desc="Çeviriler"), 1):
    try:
        start = time.time()
        translation = translate_to_turkish(sentence)
        elapsed = time.time() - start

        results.append({
            'Orijinal': sentence,
            'Çeviri': translation,
            'Süre (s)': elapsed
        })

        # Her 50 cümlede bir ilerleme raporu
        if i % 50 == 0:
            avg_time = sum(r['Süre (s)'] for r in results[-50:]) / 50
            print(f"{i}/{len(sentences)} | Ortalama süre: {avg_time:.3f}s/cümle")

    except Exception as e:
        print(f"\n⚠️ Hata (Cümle {i}): {str(e)}")
        results.append({
            'Orijinal': sentence,
            'Çeviri': f"HATA: {str(e)}",
            'Süre (s)': -1
        })

total_time = time.time() - start_total
gpu_process.terminate()  # GPU logunu durdur

# İstatistikler
success_count = len([r for r in results if r['Süre (s)'] > 0])
avg_time = total_time / success_count if success_count > 0 else 0

print("\n📊 Final Sonuçlar:")
print(f"• Toplam süre: {total_time:.2f}s")
print(f"• Başarılı çeviri: {success_count}/{len(sentences)}")
print(f"• Ortalama süre: {avg_time:.3f}s/cümle")
print(f"• Saniyede çevrilen cümle: {success_count/total_time:.2f} cümle/s")

# Sonuçları kaydetme
output_file = "m2m100_translations_500.csv"
pd.DataFrame(results).to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n💾 Çeviriler '{output_file}' olarak kaydedildi")

# GPU kullanım analizi
try:
    gpu_data = pd.read_csv('gpu_usage.log', header=None, names=['Memory', 'Utilization'])
    peak_mem = gpu_data['Memory'].max()
    avg_util = gpu_data['Utilization'].mean()
    print(f"\n🎮 GPU Kullanımı:")
    print(f"• Maksimum bellek: {peak_mem} MB")
    print(f"• Ortalama kullanım: {avg_util:.1f}%")
except Exception as e:
    print(f"\n⚠️ GPU log analiz hatası: {str(e)}")

# Model dosya boyutları
print("\n📦 Model Depolama Bilgisi:")
!du -sh ~/.cache/huggingface/hub/models--facebook--m2m100_418M

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.7 MB/s eta 0:00:00

🔍 facebook/m2m100_418M modeli yükleniyor...


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

...

400/500 | Ortalama süre: 0.365s/cümle



Çeviriler:  90%|█████████ | 450/500 [02:51<00:16,  3.10it/s]

450/500 | Ortalama süre: 0.371s/cümle



Çeviriler: 100%|██████████| 500/500 [03:08<00:00,  2.65it/s]

500/500 | Ortalama süre: 0.352s/cümle

📊 Final Sonuçlar:
• Toplam süre: 188.82s
• Başarılı çeviri: 500/500
• Ortalama süre: 0.378s/cümle
• Saniyede çevrilen cümle: 2.65 cümle/s

💾 Çeviriler 'm2m100_translations_500.csv' olarak kaydedildi

⚠️ GPU log analiz hatası: Could not convert string ' utilization.gpu [%] 21 % 32 % 44 % 44 % 43 % 28 % 11 % 19 % 62 % 60 % 62 % 60 % 18 % 19 % 61 % 41 % 25 % 44 % 46 % 55 % 43 % 63 % 59 % 63 % 66 % 67 % 67 % 68 % 63 % 69 % 64 % 54 % 48 % 43 % 64 % 64 % 63 % 64 % 62 % 63 % 62 % 69 % 57 % 60 % 44 % 38 % 55 % 44 % 64 % 64 % 63 % 60 % 64 % 63 % 62 % 63 % 64 % 61 % 50 % 54 % 46 % 61 % 66 % 63 % 57 % 63 % 62 % 59 % 63 % 63 % 64 % 48 % 45 % 54 % 55 % 64 % 65 % 68 % 66 % 68 % 67 % 63 % 65 % 63 % 44 % 43 % 56 % 43 % 58 % 62 % 63 % 58 % 59 % 57 % 57 % 69 % 63 % 62 % 52 % 53 % 46 % 68 % 67 % 68 % 68 % 63 % 60 % 70 % 62 % 63 % 62 % 43 % 38 % 49 % 44 % 62 % 61 % 67 % 65 % 68 % 67 % 63 % 62 % 67 % 67 % 49 % 48 % 45 % 67 % 61 % 63 % 68 % 54 % 58 % 56 % 59 % 62 % 68 